In [1]:
library(tidyverse)
library(jsonlite)
library(WGCNA)
library(flashClust)
library(matrixStats)

# Set number of threads
nThreads = 30

# Set directlory for storing experiment results
setwd("./")

getwd()

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘jsonlite’


The following object is masked from ‘package:purrr’:

    flatten


Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor



Attaching package: ‘flashClust’


The following object is masked from ‘package:fastcluster’:

    hclust


The following object is masked from ‘package:stats’:

    hclust



Attaching package: ‘matrixStats’


The following object i

[1] "/data/projects/robin/cell_free/proteomics_WGCNA"

In [2]:
base_dir = "/data/projects/robin/cell_free/proteomics_WGCNA/"
data = "processed/lfq.csv"
metadata = "processed/metadata.csv"

In [3]:
obj = paste(base_dir, data, sep="")
vst_mat = read.csv(obj, row.names=1)

In [4]:
obj_metadata = paste(base_dir, metadata, sep="")
anno = read.csv(obj_metadata, row.names=1)

In [5]:
vst_mat_corrected <- empiricalBayesLM(t(vst_mat), anno$batch)

In [6]:
vst_mat <- t(vst_mat_corrected[[1]])

In [7]:
anno$group <- anno[, "Neuronal.subclass..0.low..1.high."]

In [8]:
conditions<-factor(sapply(rownames(anno),function(id){paste(anno[id,c("group")],collapse = '_')}))

In [9]:
conditions

F1  F2  F3  F4  F5  F6  F7  F8  F9 F10 F11 F13 F14 F15 F16 F17 F18 F19 F20 F21 
  1   0   1   0   0   0   0   0   0   0   0   1   0   1   1   1   1   0   0   0 
F23 F24 F26 F28 F29 F30 F36 F38 
  0   0   0   0   0   1   1   1 
Levels: 0 1

In [10]:
trait_df<-data.frame(t(sapply(conditions,function(condition){table(condition)})),row.names=rownames(anno))

In [11]:
# This creates an object called "datExpr" that contains the normalized counts file output from DESeq2
datExpr = vst_mat
datExpr = t(datExpr)

In [12]:
colnames(trait_df)

[1] "X0" "X1"

In [13]:
colnames(trait_df) <- c("low", "high")

In [14]:
datTraits = trait_df
table(rownames(datTraits)==rownames(datExpr))


TRUE 
  28 

In [15]:
# Cluster samples by expression ----------------------------------------------------------------

A = adjacency(t(datExpr),type="signed") # this calculates the whole network connectivity
k = as.numeric(apply(A,2,sum))-1 # standardized connectivity
Z.k = scale(k)
thresholdZ.k = -2.5 # often -2.5
outlierColor = ifelse(Z.k<thresholdZ.k,"red","black")
sampleTree = flashClust(as.dist(1-A), method = "average")
# Convert traits to a color representation where red indicates high values
traitColors = data.frame(numbers2colors((datTraits),signed=FALSE))
dimnames(traitColors)[[2]] = paste(names(datTraits))
datColors = data.frame(outlier = outlierColor,traitColors)

png(filename="sampleDendorgramAndTraitHeatmap.png")
plotDendroAndColors(sampleTree,groupLabels=names(datColors),
                    colors=datColors,main="Sample Dendrogram and Trait Heatmap")
dev.off()

png 
  2

**remove outliers**

In [16]:
outliers <- c("F17")

In [17]:
gsg = goodSamplesGenes(datExpr, verbose = 3);
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1


[1] TRUE

In [18]:
datTraits <- datTraits[!(row.names(datTraits) %in% outliers),]
datExpr <- datExpr[!(row.names(datExpr) %in% outliers),]

In [19]:
table(rownames(datTraits)==rownames(datExpr))


TRUE 
  27 

In [20]:
# Choose a soft threshold power
powers = c(c(1:10), seq(from =10, to=100, by=0.5)) #choosing a set of soft-thresholding powers
sft = pickSoftThreshold(datExpr, dataIsExpr = TRUE, powerVector=powers, 
                        verbose =5, networkType="signed",
                        corOptions = list(use = 'p')) #call network topology analysis function

sizeGrWindow(9,5)
par(mfrow= c(1,2))
cex1=0.9

pickSoftThreshold: will use block size 4716.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 4716 of 4716


Warning message:
“executing %dopar% sequentially: no parallel backend registered”


    Power SFT.R.sq    slope truncated.R.sq mean.k. median.k. max.k.
1     1.0 5.06e-01  5.16000          0.892  2670.0  2.67e+03   3210
2     2.0 1.66e-01  1.37000          0.590  1610.0  1.58e+03   2310
3     3.0 5.10e-07  0.00144          0.385  1020.0  9.69e+02   1740
4     4.0 1.55e-01 -0.65400          0.231   674.0  6.14e+02   1370
5     5.0 3.73e-01 -1.09000          0.262   466.0  3.99e+02   1120
6     6.0 4.72e-01 -1.37000          0.323   334.0  2.66e+02    940
7     7.0 5.72e-01 -1.54000          0.450   249.0  1.82e+02    813
8     8.0 5.73e-01 -1.60000          0.460   191.0  1.26e+02    721
9     9.0 6.42e-01 -1.57000          0.570   151.0  8.87e+01    652
10   10.0 6.07e-01 -1.59000          0.561   123.0  6.32e+01    599
11   10.0 6.07e-01 -1.59000          0.561   123.0  6.32e+01    599
12   10.5 5.83e-01 -1.59000          0.552   112.0  5.38e+01    577
13   11.0 5.52e-01 -1.59000          0.535   103.0  4.57e+01    558
14   11.5 5.28e-01 -1.57000          0.537    94

In [21]:
png("scaleIndependence.png")
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2], xlab= "Soft Threshold (power)", 
     ylab="Scale Free Topology Model Fit, signed R^2", type= "n", main= paste("Scale independence"))
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2], labels=powers, cex=cex1, col="red")
abline(h=0.80, col="red")
dev.off()

png("meanConnectivity.png")
plot(sft$fitIndices[,1], sft$fitIndices[,5], xlab= "Soft Threshold (power)", ylab="Mean Connectivity", type="n", 
     main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1, col="red")
dev.off()

png 
  2

png 
  2

In [22]:
softPower <- 9

In [23]:
nThreads <- 50
enableWGCNAThreads(nThreads=nThreads)
adjacency = adjacency(datExpr, power = softPower, type = "signed") #specify network type


Allowing parallel execution with up to 50 working processes.


In [24]:
# Construct Networks- USE A SUPERCOMPUTER IRL -----------------------------
  
#translate the adjacency into topological overlap matrix and calculate the corresponding dissimilarity:
TOM = TOMsimilarity(adjacency, TOMType="signed") # specify network type
dissTOM = 1-TOM
  
# Generate Modules --------------------------------------------------------
  
# Generate a clustered gene tree
geneTree = flashClust(as.dist(dissTOM), method="average")

png("geneClusteringTOM.png")
plot(geneTree, xlab="", sub="", main= "Gene Clustering on TOM-based dissimilarity", labels= FALSE, hang=0.04)
dev.off()

..connectivity..
..matrix multiplication (system BLAS)..
..normalization..
..done.


png 
  2

In [25]:
packageVersion("clusterProfiler")

[1] ‘4.2.0’

In [26]:
#This sets the minimum number of genes to cluster into a module
minModuleSize = 10
x = 4 
dynamicMods = cutreeDynamic(dendro = geneTree, distM = as.matrix(dissTOM), 
                            method="hybrid", pamStage = F, deepSplit = x, 
                            minClusterSize = minModuleSize)
  
dynamicColors= labels2colors(dynamicMods)
MEList= moduleEigengenes(datExpr, colors= dynamicColors)#,softPower = 14)
MEs= MEList$eigengenes
MEDiss= 1-cor(MEs)
METree= flashClust(as.dist(MEDiss), method= "average")
  
save(dynamicMods, MEList, MEs, MEDiss, METree, file= "Network_allSamples_signed_RLDfiltered.RData")
  
#plots tree showing how the eigengenes cluster together
png(paste("eigenGenesClustering", ".png", sep=""))
plot(METree, main= "Clustering of module eigengenes", xlab= "", sub= "")
dev.off()

#softPower = 10 # see scale independence file "softthreshold_SOD1.png"

 ..cutHeight not given, setting it to 0.983  ===>  99% of the (truncated) height range in dendro.
 ..done.


png 
  2

In [27]:
#### -----------------------------------------------------------------------
#set a threhold for merging modules. In this example we are not merging so MEDissThres=0.0
MEDissThres = 0.4
merge = mergeCloseModules(datExpr, dynamicColors, cutHeight= MEDissThres, verbose =3)
mergedColors = merge$colors
mergedMEs = merge$newMEs

#plot dendrogram with module colors below it
png("geneTree.png")
plotDendroAndColors(geneTree, cbind(dynamicColors, mergedColors), c("Dynamic Tree Cut", "Merged dynamic"), dendroLabels= FALSE, hang=0.03, addGuide= TRUE, guideHang=0.05)
dev.off()
moduleColors = mergedColors
colorOrder = c("grey", standardColors(50))
moduleLabels = match(moduleColors, colorOrder)-1
MEs = mergedMEs

save(MEs, moduleLabels, moduleColors, geneTree, file= "Network_allSamples_signed_nomerge_RLDfiltered.RData")

 mergeCloseModules: Merging modules whose distance is less than 0.4
   multiSetMEs: Calculating module MEs.
     Working on set 1 ...
     moduleEigengenes: Calculating 30 module eigengenes in given set.
   multiSetMEs: Calculating module MEs.
     Working on set 1 ...
     moduleEigengenes: Calculating 20 module eigengenes in given set.
   multiSetMEs: Calculating module MEs.
     Working on set 1 ...
     moduleEigengenes: Calculating 19 module eigengenes in given set.
   Calculating new MEs...
   multiSetMEs: Calculating module MEs.
     Working on set 1 ...
     moduleEigengenes: Calculating 19 module eigengenes in given set.


png 
  2

In [28]:
#Define number of genes and samples
nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

#Recalculate MEs with color labels
MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use="p")

write.csv(moduleTraitCor, "moduleTraitCor.csv")

moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples)

write.csv(moduleTraitPvalue, "moduleTraitPvalue.csv")

In [29]:
#display the corelation values with a heatmap plot
par(mar=  c(6, 9, 3, 3))
png("moduleTraitRelationshipHeatmap_corrAll.png", width=600, height=600)
labeledHeatmap(Matrix= t(moduleTraitCor), 
               yLabels= names(datTraits), 
               xLabels= names(MEs),
               xSymbols= names(MEs),
               colorLabels= FALSE, 
               colors= blueWhiteRed(50), 
               #textMatrix= textMatrix[module_selection,], 
               setStdMargins= TRUE, 
               cex.text= 0.5, 
               zlim= c(-1,1), 
               main= paste("Module-trait relationships"))
dev.off()

png 
  2

In [30]:
#get selected genes
selected_Colors<-sapply(names(MEs),function(ME){sub("ME","",ME)})
#names(datExpr)[moduleColors=="brown"]
module_genes<-sapply(selected_Colors,function(module_color){rownames(t(datExpr)[mergedColors==module_color,])})

WGCNA_folder<-"./"
saveRDS(module_genes,file=file.path(WGCNA_folder,'module_genes.rds'))
write(toJSON(module_genes),file=file.path(WGCNA_folder,'module_genes.json'))

## enrichment of each module
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

BiocManager::install("pathview")

suppressPackageStartupMessages({
  library(clusterProfiler)
  library(enrichplot)
  library(DOSE)
  library(org.Mm.eg.db)
  library(org.Hs.eg.db)
  library(ggplot2)
  library(pathview)
  library(tidyverse)
})

make_go_enr = function(genelist, 
                       fname, 
                       folder, 
                       fromType="SYMBOL",
                       #OrgDb=org.Mm.eg.db,
                       OrgDb=org.Hs.eg.db,
                       prefixes=c("BP","MF","CC","All"),
                       image_width=30,
                      image_height=30){
  
  gene.df <- bitr((genelist), fromType = fromType,
                  toType = c("ENTREZID"),
                  OrgDb = OrgDb)
  for (prefix in prefixes) {
    ego <- enrichGO(gene          = gene.df$ENTREZID,
                    OrgDb         = OrgDb,
                    ont           = prefix,
                    pAdjustMethod = "BH",
                    pvalueCutoff  = 0.05,
                    qvalueCutoff  = 0.05,
                    readable = T)
    
    dir.create(file.path(folder, fname))
    
    if (!is.null(ego) && nrow(ego)!=0){
      write.csv(ego, file.path(folder, fname, paste0("go_enr_", prefix,"_",fname,".csv")))

      p1 = dotplot(ego, showCategory=20) + ggtitle(paste0("GO enrichment ", prefix))
      ggsave(p1, filename = file.path(folder,fname, paste0("dotplot_go_", prefix,"_",fname,".pdf")), width = image_width, 
             height=image_height, units="cm")

      p2 = barplot(ego, showCategory=20) + ggtitle(paste0("GO enrichment ", prefix))
      ggsave(p2, filename = file.path(folder, fname, paste0("barplot_go_", prefix,"_",fname,".pdf")), width = image_width, 
             height=image_height, units="cm")
    }
  }
}

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://packagemanager.rstudio.com/all/__linux__/focal/2021-10-18+Y3JhbjoyMDIxLTEwLTE1LDI6NDUyNjIxNTsxMzE0RjdBNg


Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Warning message:
“package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'pathview'”
Installation paths not writeable, unable to update packages
  path: /usr/local/lib/R/library
  packages:
    lattice, mgcv, nlme, survival

Old packages: 'basilisk', 'basilisk.utils', 'BiocFileCache', 'BiocParallel',
  'biomaRt', 'ChIPseeker', 'clusterProfiler', 'cytolib', 'dir.expiry',
  'enrichplot', 'ensembldb', 'GeneTonic', 'GenomeInfoDb', 'GenomicFeatures',
  'limma', 'MassSpecWavelet', 'quantiseqr', 'rhdf5', 'S4Vectors', 'singscore'



In [49]:
make_go_enr1 = function(genelist, 
                       fname, 
                       folder, 
                       fromType="SYMBOL",
                       #OrgDb=org.Mm.eg.db,
                       OrgDb=org.Hs.eg.db,
                       prefixes=c("BP","MF","CC","All"),
                       image_width=20,
                        image_height=20){
  
  gene.df <- bitr((genelist), fromType = fromType,
                  toType = c("ENTREZID"),
                  OrgDb = OrgDb)
  for (prefix in prefixes) {
    ego <- enrichGO(gene          = gene.df$ENTREZID,
                    OrgDb         = OrgDb,
                    ont           = prefix,
                    pAdjustMethod = "BH",
                    pvalueCutoff  = 0.05,
                    qvalueCutoff  = 0.05,
                    readable = T)
    
    dir.create(file.path(folder, fname))
    
    if (!is.null(ego) && nrow(ego)!=0){
      write.csv(ego, file.path(folder, fname, paste0("go_enr_", prefix,"_",fname,".csv")))

      p1 = dotplot(ego, showCategory=10, font.size=20) + ggtitle(paste0("GO enrichment ", prefix))
      ggsave(p1, filename = file.path(folder,fname, paste0("dotplot_go_", prefix,"_",fname,".pdf")), width = image_width, 
             height=image_height, units="cm")

      p2 = barplot(ego, showCategory=10, font.size=20) + ggtitle(paste0("GO enrichment ", prefix))
      ggsave(p2, filename = file.path(folder, fname, paste0("barplot_go_", prefix,"_",fname,".pdf")), width = image_width, 
             height=image_height, units="cm")
    }
  }
}

In [50]:
modules_ <- c("MEblue", "MEbrown")

In [51]:
result_folder<-"results_MEblue_MEbrown"
overview_df<- data.frame()

dir.create(result_folder)

for (module in modules_){ 

  make_go_enr1(module_genes[[module]],module,file.path(WGCNA_folder,result_folder))
}

Warning message in dir.create(result_folder):
“'results_MEblue_MEbrown' already exists”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr((genelist), fromType = fromType, toType = c("ENTREZID"), :
“1.67% of input gene IDs are fail to map...”
Warning message in dir.create(file.path(folder, fname)):
“'.//results_MEblue_MEbrown/MEblue' already exists”
Warning message in dir.create(file.path(folder, fname)):
“'.//results_MEblue_MEbrown/MEblue' already exists”
Warning message in dir.create(file.path(folder, fname)):
“'.//results_MEblue_MEbrown/MEblue' already exists”
Warning message in dir.create(file.path(folder, fname)):
“'.//results_MEblue_MEbrown/MEblue' already exists”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr((genelist), fromType = fromType, toType = c("ENTREZID"), :
“6.02% of input gene IDs are fail to map...”
Warning message in dir.create(file.path(folder, fname)):
“'.//results_MEblue_MEbrown/MEbrown' already ex

In [37]:
result_folder<-"results"
overview_df<- data.frame()

dir.create(result_folder)

for (module in names(module_genes)){ 

  make_go_enr(module_genes[[module]],module,file.path(WGCNA_folder,result_folder))
}

Warning message in dir.create(result_folder):
“'results' already exists”
'select()' returned 1:1 mapping between keys and columns

Warning message in dir.create(file.path(folder, fname)):
“'.//results/MEdarkorange' already exists”
Warning message in dir.create(file.path(folder, fname)):
“'.//results/MEdarkorange' already exists”
Warning message in dir.create(file.path(folder, fname)):
“'.//results/MEdarkorange' already exists”
'select()' returned 1:1 mapping between keys and columns

Warning message in dir.create(file.path(folder, fname)):
“'.//results/MEdarkturquoise' already exists”
Warning message in dir.create(file.path(folder, fname)):
“'.//results/MEdarkturquoise' already exists”
Warning message in dir.create(file.path(folder, fname)):
“'.//results/MEdarkturquoise' already exists”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr((genelist), fromType = fromType, toType = c("ENTREZID"), :
“2.22% of input gene IDs are fail to map...”
Warning message 

In [ ]:
# modules_ <- c("MEblue", "MEbrown")

In [ ]:
# result_folder<-"results"
# overview_df<- data.frame()
# 
# dir.create(result_folder)
# 
# for (module in modules_){ # names(module_genes)
# 
#   make_go_enr(module_genes[[module]],module,file.path(WGCNA_folder,result_folder))
# }

In [ ]:
# modules_ <- c("MEroyalblue", "MEmagenta")

In [ ]:
# result_folder<-"results"
# overview_df<- data.frame()

# dir.create(result_folder)

# for (module in modules_){ # names(module_genes)

#   make_go_enr(module_genes[[module]],module,file.path(WGCNA_folder,result_folder))
# }